

Cody's version.
I need names and signatures

Maturin

Alternative approach: write lean code to generate python code? Traverse module, find exports, convert to equivalent C versions.

nanobind using cpp export?



In [ ]:
def lean_code():

In [ ]:
import cffi
class LeanModuleGen():
    def __init__(self, name):
        self.name = name
        self.ffi = cffi.FFI()
        self.ffi.cdef("""
            typedef struct lean_object;
            void lean_runtime_init();
        """)
    def lean_io_mk_world():
    def lean_dec():
    def lean_inc():
    def lean_unbox():
    def lean_box(): 
    def cdef(x):
        self.ffi.cdef(x)
    def compile(self, code):
    def load(self, dylib):


In [ ]:
import subprocess
import cffi
def compile_lean(code):
    with open("/tmp/mylean.lean", "w") as f:
        f.write(code)
        f.flush()
    leanlibpath = subprocess.run(["lean", "--print-libdir"], capture_output=True, text=True).stdout.strip()
    subprocess.run(["lean", "/tmp/mylean.lean", "-c", "/tmp/mylean.c"])
    subprocess.run(["leanc", "-c", "-fPIC", "-o", "/tmp/mylean.o", "/tmp/mylean.c", "-DLEAN_EXPORTING"])
    subprocess.run(["leanc", "-shared", "-o", "/tmp/mylean.so", 
        "/tmp/mylean.o", "-lleanshared_1", "-lleanshared",
        "-Wl,-rpath=" + leanlibpath])

code = """


inductive Testo where
    | mk : Nat → Testo

@[export my_add]
def my_add (x y : UInt64) : UInt64 :=
    x + y

"""


def compile_and_load_lean(code):
    compile_lean(code)
    ffi = cffi.FFI()
    # universal stuff
    ffi.cdef("""
        typedef struct lean_object;
        typedef lean_object * lean_obj_arg;  
    """)
    ffi.cdef("""
        uint64_t my_add(uint64_t, uint64_t);
    """)
    return ffi.dlopen("/tmp/mylean.so")
    
    #lean.my_add(1, 2)  # should return 3


lean = compile_and_load_lean(code)
lean.my_add(1,2)

CDefError: cannot parse "typedef lean_object * lean_obj_arg;"
<cdef source string>:3:29: before: *

https://github.com/dselsam/lean-python-bindings

https://github.com/leanprover/lean4/tree/master/src/lake/examples/reverse-ffi

https://docs.lean-lang.org/lean4/doc/dev/ffi.html#borrowing

https://github.com/leanprover/lean4/tree/master/tests/compiler/foreign

https://github.com/digama0/lean-sys 
Can i emscripten compile? Do DSL parser in lean.

    libInit.a libLean.a  # static libraries of the Lean library
    libleancpp.a  # a static library of the C++ sources of Lean
    libleanshared.so  # a dynamic library including the static libraries above


https://proofassistants.stackexchange.com/questions/4851/how-do-you-compile-and-link-a-lean-program-by-hand

https://lean-lang.org/papers/lean4.pdf section 3 code generator
@specialize and @inline




- Lean.Declaration -0 different kinds of dewclarations tyo kernel. Hmmm.
- Lean.CoreM
- Lean.Envrionment - THis seems like the Kernel interface? Environment extesnsions?
addDeclCore. Lean.Kernel.Environment. Lean.Enrvionement is wrapper.
importModules
evalConst


- Lean.Expr
- Lean.LoadDynLib

- Lean.Elab https://leanprover-community.github.io/mathlib4_docs/Lean/Elab/Frontend.html#Lean.Elab.runFrontend

lean plugins vs dybnlibs

https://github.com/tydeu/lean4-alloy interesting. Embed C code inside lean files



Using lean as a solver


In [6]:
! ls $(lean --print-prefix)/include/lean

config.h  lean_gmp.h  lean.h  lean_libuv.h  version.h


libleanshared.so and libleanshared_1.so. I'm not sure what the _1.so does? There really isn't much in there. The bulk of stuff seems to be in the first.
https://lean-lang.org/doc/reference/latest/releases/v4.12.0/?utm_source=chatgpt.com Ah, ok so maybe it's a workaround for Windows having a symbol limit, hence on my linux system it'as a null library


lbInit_shared.so is a compiled version of the lean code in Init

- The .a are stat8ic archive
- .so is shared object
- .ilean and .olean are lean stuff. I dunno what format they are



libleanshared seems like it contains the entire Lean subproject, which makes sense. Lots of stuff has mangled-ish names. probably nothing is really labelled for @export.
It also seems to have Init and Std in it on my system. 

https://github.com/leanprover/lean4/tree/master/src/kernel

https://github.com/leanprover/lean4/blob/master/src/initialize/init.cpp wherea lean_initialize is.


In [29]:
! objdump -x $(lean --print-libdir)/libleanshared.so | grep runFront

00000000074a50f0 l     O .bss	0000000000000008              l_Lean_Elab_runFrontend___lambda__4___closed__1
00000000074a50f8 l     O .bss	0000000000000008              l_Lean_Elab_runFrontend___lambda__4___closed__2
00000000074a5100 l     O .bss	0000000000000008              l_Lean_Elab_runFrontend___lambda__5___closed__1
00000000074a5108 l     O .bss	0000000000000008              l_Lean_Elab_runFrontend___closed__1
00000000074a5110 l     O .bss	0000000000000008              l_Lean_Elab_runFrontend___closed__2
00000000074a5118 l     O .bss	0000000000000008              l_Lean_Elab_runFrontend___closed__4
00000000074a5120 l     O .bss	0000000000000008              l_Lean_Elab_runFrontend___closed__3
0000000002f57470 g     F .text	00000000000000b4              l_Lean_Option_setIfNotSet___at_Lean_Elab_runFrontend___spec__1
0000000002f57530 g     F .text	0000000000000149              l_Array_mapMUnsafe_map___at_Lean_Elab_runFrontend___spec__2
0000000002f57680 g     F .text	00000000000001d1

In [1]:
import cffi
import subprocess
ffi = cffi.FFI()
ffi.cdef("""
typedef struct lean_object;
void lean_initialize();
""")
lean_libdir = subprocess.check_output(["lean", "--print-libdir"]).decode().strip()
lean = ffi.dlopen(lean_libdir + "/libleanshared.so")
lean.lean_initialize() 



In [10]:
#ffi.new("struct lean_object *")
def lean_box(n):
    return ffi.cast("struct lean_object*", (n << 1) | 1)
def lean_unbox(obj):
    return int(ffi.cast("uintptr_t", obj)) >> 1
def lean_io_mk_world():
    return lean_box(0)
lean_io_mk_world()

<cdata 'struct lean_object *' 0x1>

In [7]:
! ls $(lean --print-libdir)

Init		Lake.olean	   libLake_shared.so   libStd.a
Init.ilean	Lean		   libLean.a	       Std
Init.olean	Lean.ilean	   libleancpp.a        Std.ilean
Lake		Lean.olean	   libleanmanifest.a   Std.olean
Lake.ilean	libInit.a	   libleanrt.a
LakeMain.ilean	libInit_shared.so  libleanshared_1.so
LakeMain.olean	libLake.a	   libleanshared.so


In [5]:
%%file /tmp/hello.lean

@[export hello]
def hello : String :=
  "Hello"

@[export my_length]
def myLength (s : String) : UInt64 :=
  s.length.toUInt64


Overwriting /tmp/hello.lean


In [16]:
! lean /tmp/hello.lean  --profile

cumulative profiling times:
	attribute application 0.0431ms
	compilation 0.489ms
	elaboration 1ms
	fix level params 0.00801ms
	import 75.7ms
	initialization 25.4ms
	instantiate metavars 0.01ms
	interpretation 1.84ms
	linting 0.153ms
	parsing 0.367ms
	process pre-definitions 0.131ms
	share common exprs 0.0122ms
	type checking 0.0389ms


https://leanprover-community.github.io/lean4-metaprogramming-book/extra/01_options.html
-D let's you set options from the cli. Thats interesting.

lcnf is function pres-ssa. lean compiler normal form.  https://leanprover-community.github.io/mathlib4_docs/Lean/Compiler/LCNF/Basic.html based on A normal form https://www.microsoft.com/en-us/research/wp-content/uploads/2016/11/compiling-without-continuations.pdf https://leanprover-community.github.io/mathlib4_docs/Lean/Compiler/LCNF/Types.html
jp = join point



In [28]:
! lean /tmp/hello.lean -Dtrace.Compiler.simp=true -Dtrace.Meta.synthInstance=true  -Dtrace.compiler.ir.result=true  -Dtrace.compiler.lcnf=true -Dtrace.Meta.isDefEq=true

[Meta.isDefEq] ✅️ Sort ?u.1 =?= Type
[Meta.isDefEq] ✅️ String =?= String
[compiler.lcnf] >> hello
"Hello"

[result]
def hello._closed_1 : obj :=
  let x_1 : obj := "Hello";
  ret x_1
def hello : obj :=
  let x_1 : obj := hello._closed_1;
  ret x_1
[Meta.isDefEq] ✅️ Sort ?u.3 =?= Type
[Meta.isDefEq] ✅️ Sort ?u.5 =?= Type
[Meta.isDefEq] ✅️ String =?= String
  [Meta.isDefEq] ✅️ String =?= String
[Meta.isDefEq] ✅️ Nat =?= Nat
  [Meta.isDefEq] ✅️ Nat =?= Nat
[Meta.isDefEq] ✅️ UInt64 =?= UInt64
[Meta.isDefEq] ✅️ UInt64 =?= UInt64
[compiler.lcnf] >> myLength
fun s =>
  let _x_1 := s.length;
  _x_1.toUInt64

[result]
def myLength (x_1 : obj) : u64 :=
  let x_2 : obj := String.length x_1;
  dec x_1;
  let x_3 : u64 := UInt64.ofNat x_2;
  dec x_2;
  ret x_3
def myLength._boxed (x_1 : obj) : obj :=
  let x_2 : u64 := myLength x_1;
  let x_3 : obj := box x_2;
  ret x_3


LEAN_EXPORTING to set visibility to default so variables aren't hidden?

In [21]:
! cd /tmp && lean -c /tmp/hello.c  /tmp/hello.lean && leanc -o /tmp/hello.o -c /tmp/hello.c -DLEAN_EXPORTING

In [7]:
! cat /tmp/hello.c

// Lean compiler output
// Module: hello
// Imports: Init
#include <lean/lean.h>
#if defined(__clang__)
#pragma clang diagnostic ignored "-Wunused-parameter"
#pragma clang diagnostic ignored "-Wunused-label"
#elif defined(__GNUC__) && !defined(__CLANG__)
#pragma GCC diagnostic ignored "-Wunused-parameter"
#pragma GCC diagnostic ignored "-Wunused-label"
#pragma GCC diagnostic ignored "-Wunused-but-set-variable"
#endif
#ifdef __cplusplus
extern "C" {
#endif
uint64_t lean_uint64_of_nat(lean_object*);
LEAN_EXPORT lean_object* hello;
static lean_object* l_hello___closed__1;
LEAN_EXPORT lean_object* l_myLength___boxed(lean_object*);
lean_object* lean_string_length(lean_object*);
LEAN_EXPORT uint64_t my_length(lean_object*);
static lean_object* _init_l_hello___closed__1() {
_start:
{
lean_object* x_1; 
x_1 = lean_mk_string_unchecked("Hello", 5, 5);
return x_1;
}
}
static lean_object* _init_hello() {
_start:
{
lean_object* x_1; 
x_1 = l_hello___closed__1;
return x_1;
}
}
LEAN_EXPORT uint64_t m

'/home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean'

In [22]:
! leanc -shared -o /tmp/hello.so /tmp/hello.o  -L $(lean --print-libdir) -lInit_shared -lleanshared_1 -lleanshared -I  -l /lib/x86_64-linux-gnu/libmimalloc.so -Wl,-rpath=$(lean --print-libdir) 

In [24]:
! gcc -shared -o /tmp/hello.so /tmp/hello.o   -L $(lean --print-libdir) -lInit_shared -lleanshared_1 -lleanshared -lmimalloc -Wl,-rpath=$(lean --print-libdir) 

In [14]:
! gcc -shared -o /tmp/hello.so /tmp/hello.o   -L $(lean --print-libdir) -Wl,-rpath=$(lean --print-libdir) 

In [12]:
import subprocess
lean_prefix = subprocess.check_output(["lean", "--print-prefix"]).decode().strip()
lean_prefix

with open(lean_prefix + "/include/lean/lean.h", "r") as f:
    lean_h_content = f.read()
    with open("/tmp/leandummy.c", "w") as f1:
        f1.write(lean_h_content.replace("static inline ", ""))


In [20]:
! gcc -fPIC -c -o /tmp/leandummy.o /tmp/leandummy.c -I $(lean --print-prefix)/include && gcc -shared -o /tmp/leandummy.so /tmp/leandummy.o -lleanshared_1 -lleanshared  -L $(lean --print-libdir) -Wl,-rpath=$(lean --print-libdir)

/tmp/leandummy.c:7:9: warning: #pragma once in main file
    7 | #pragma once
      |         ^~~~


In [22]:
import cffi
ffi = cffi.FFI()
ffi.cdef("""
typedef struct lean_object;
struct lean_object * lean_io_mk_world();
""")
lib = ffi.dlopen("/tmp/leandummy.so")
lib.lean_io_mk_world()

<cdata 'struct lean_object *' 0x1>

Ok, But this is crazy. I probably don't have to shim that much stuff,

In [44]:
%%file /tmp/leanshim.c

#include <lean/lean.h>

lean_obj_res leanshim_io_mk_world() {
    return lean_io_mk_world();
}

char const * leanshim_string_cstr(b_lean_obj_arg o) {
    return lean_string_cstr(o);
}


Overwriting /tmp/leanshim.c


In [45]:
! gcc /tmp/leanshim.c -fPIC -c -o /tmp/leanshim.o -I $(lean --print-prefix)/include && gcc -shared -o /tmp/leanshim.so /tmp/leanshim.o -lleanshared_1 -lleanshared  -L $(lean --print-libdir) -Wl,-rpath=$(lean --print-libdir)

In [ ]:
import cffi
ffi = cffi.FFI()
ffi.cdef("""
typedef struct lean_object;
typedef struct len_obj_res;
struct lean_object * leanshim_io_mk_world();
""")
shim = ffi.dlopen("/tmp/leanshim.so")
shim.leanshim_io_mk_world()

<cdata 'struct lean_object *' 0x1>

In [3]:
import cffi
ffi = cffi.FFI()
ffi.cdef("""
typedef struct lean_object;
typedef struct len_obj_res;
struct lean_object * leanshim_io_mk_world();
char* leanshim_string_cstr(struct lean_object*);
""")
shim = ffi.dlopen("/tmp/leanshim.so")
shim.leanshim_io_mk_world()
#ffi = cffi.FFI()
ffi.cdef("""
typedef struct lean_object;
extern struct lean_object * hello;
uint64_t my_length(struct lean_object*);
void lean_initialize_runtime_module();
""")
ffi.cdef("struct lean_object * initialize_hello(uint8_t builtin, struct lean_object *);")

lib = ffi.dlopen("/tmp/hello.so")
lib.lean_initialize_runtime_module()
lib.initialize_hello(0, shim.leanshim_io_mk_world())
ffi.string(shim.leanshim_string_cstr(lib.hello))
lib.my_length(lib.hello)

5

Design choices

- nanobind (c++ only?)
- cffi abi mode
- cffi api mode

inline or out of line

Do I bind everything in lean.h? Meh.



What is leanmake for? Just to have CC set to leanc?


src/runtime
vs the expression datatype.



In [7]:
import subprocess
lean_prefix = subprocess.check_output(["lean", "--print-prefix"]).decode().strip()
lean_prefix
lean_libdir = subprocess.check_output(["lean", "--print-libdir"]).decode().strip()
lean_libdir

import cffi
ffi = cffi.FFI()
ffi.cdef("""
typedef struct lean_object;
struct lean_object * lean_io_mk_world();
""")
lib = ffi.dlopen(lean_libdir + "/libleanshared.so")
lib.lean_io_mk_world()

AttributeError: function/symbol 'lean_io_mk_world' not found in library '/home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean/libleanshared.so': /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean/libleanshared.so: undefined symbol: lean_io_mk_world

In [5]:
import cffi
ffi = cffi.FFI()
ffi.cdef("""
typedef struct lean_object;
extern struct lean_object * hello;
uint64_t my_length(struct lean_object*);
void lean_initialize_runtime_module();
struct lean_object * lean_io_mk_world();

""")

lib = ffi.dlopen("/tmp/hello.so")
lib.lean_initialize_runtime_module()
lib.hello

lib.lean_io_mk_world()
#lib.lean_box(0)

AttributeError: function/symbol 'lean_io_mk_world' not found in library '/tmp/hello.so': /tmp/hello.so: undefined symbol: lean_io_mk_world

In [3]:
import cffi
ffi = cffi.FFI()
ffi.cdef("uint64_t initialize_myadd();")
ffi.cdef("void lean_initialize_runtime_module();")
ffi.cdef("uint64_t myadd(uint64_t a, uint64_t b);")
lib = ffi.dlopen("/tmp/myadd.so")
#lib.initialize_myadd()
lib.initialize_lean_runtime_module()
print(lib.myadd(5, 7))

OSError: cannot load library '/tmp/myadd.so': /tmp/myadd.so: undefined symbol: initialize_Init.  Additionally, ctypes.util.find_library() did not manage to locate a library called '/tmp/myadd.so'

In [4]:
with open("/proc/self/maps") as f:
    for line in f:
        if ".so" in line:
            print(line.strip())


7db83ca00000-7db83dc17000 r--p 00000000 103:05 45388998                  /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean/libleanshared.so
7db83dc17000-7db843e56000 r-xp 01216000 103:05 45388998                  /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean/libleanshared.so
7db843e56000-7db843e60000 r--p 07454000 103:05 45388998                  /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean/libleanshared.so
7db843e60000-7db843e62000 rw-p 0745d000 103:05 45388998                  /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean/libleanshared.so
7db852f8a000-7db852f8f000 r--p 00000000 103:05 87039982                  /usr/lib/x86_64-linux-gnu/libmimalloc.so.2.1
7db852f8f000-7db852f9d000 r-xp 00005000 103:05 87039982                  /usr/lib/x86_64-linux-gnu/libmimalloc.so.2.1
7db852f9d000-7db852fa2000 r--p 00013000 103:05 87039982                  /usr/lib/x86_64-linux-gnu/libmimalloc.so.2.1
7db852fa2000-7db85

In [ ]:
import cffi

ffi = cffi.FFI()


ffi.cdef("void lean_initialize_runtime_module();")
ffi.cdef("void lean_io_mark_end_initialization();")
ffi.cdef("lean_object * lean_io_mk_world();")
ffi.cdef("lean_object * initialize_hello(uint8_t builtin, lean_object *);")

lib = ffi.dlopen("liblean.so")



CDefError: cannot parse "typedef lean_object;"
<cdef source string>:1:9: Missing type in declaration

In [58]:
%%file /tmp/leanffi.c
#include <stdio.h>
#include <lean/lean.h>
uint64_t my_length(lean_obj_arg);

extern void lean_initialize_runtime_module();
//extern void lean_initialize();
extern void lean_io_mark_end_initialization();
extern lean_object * initialize_hello(uint8_t builtin, lean_object *);

int main() {
  lean_initialize_runtime_module();
  uint8_t builtin = 1;
  lean_object* res = initialize_hello(builtin, lean_io_mk_world());

  
  lean_object * s = lean_mk_string("hello!");
  uint64_t l = my_length(s);
  printf("output: %ld\n", l);


  lean_io_mark_end_initialization();

}

Overwriting /tmp/leanffi.c


In [57]:
! leanc /tmp/hello.o /tmp/leanffi.c -lmimalloc

/tmp/leanffi.c:1:10: fatal error: 'stdio.h' file not found
#include <stdio.h>
         ^~~~~~~~~
1 error generated.


In [34]:
! leanc -xc -E -v -

/home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/bin/clang -I /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/include -fstack-clash-protection -fwrapv -fPIC -fvisibility=hidden --sysroot /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0 -nostdinc -isystem /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/include/clang -xc -E -v - --sysroot /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0 -L /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib -L /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/glibc /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/glibc/libc_nonshared.a /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/glibc/libpthread_nonshared.a -Wl,--as-needed -Wl,-Bstatic -lgmp -lunwind -luv -Wl,-Bdynamic -Wl,--no-as-needed -fuse-ld=lld -L /home/philip/.elan/toolchains/leanprover--lean4---v4.18.0/lib/lean -Wl,--start-group -lleancpp -lLean -Wl,--end-group -lStd -Wl,--start-group -lInit -llean

The -Wl,-rpath stuff adds libs to rpath in bianry so I don't have to LD_LIBRARY_PATH 

In [72]:
! gcc /tmp/hello.o /tmp/leanffi.c -o /tmp/leanffi -I $(lean --print-prefix)/include -L $(lean --print-libdir) -lInit_shared -lleanshared_1 -lleanshared -lmimalloc -Wl,-rpath=$(lean --print-libdir) 

In [61]:
!/tmp/leanffi

output: 6


In [85]:
! clang /tmp/hello.o /tmp/leanffi.c -o /tmp/leanffi -I $(lean --print-prefix)/include -L $(lean --print-libdir) -Wl,--start-group -lleancpp -lLean -Wl,--end-group -lStd -Wl,--start-group -lInit -lleanrt -Wl,--end-group  -lc++ -lc++abi -lmimalloc -Wl,-rpath=$(lean --print-libdir)

/usr/bin/ld: cannot find -lc++: No such file or directory
/usr/bin/ld: cannot find -lc++abi: No such file or directory
clang: error: linker command failed with exit code 1 (use -v to see invocation)


In [27]:
! cd /tmp && lake new rffi

In [86]:
! cd /tmp/rffi && lake build -v

info: rffi: no previous manifest, creating one from scratch
info: toolchain not updated; already up-to-date
ℹ [2/8] Built Rffi.Basicre)more)
trace: .> LEAN_PATH=/tmp/rffi/.lake/build/lib/lean /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1/bin/lean /tmp/rffi/Rffi/Basic.lean -R /tmp/rffi -o /tmp/rffi/.lake/build/lib/lean/Rffi/Basic.olean -i /tmp/rffi/.lake/build/lib/lean/Rffi/Basic.ilean -c /tmp/rffi/.lake/build/ir/Rffi/Basic.c --json
ℹ [3/8] Built Rffi.Basic:c.o (with exports)re)
trace: .> /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1/bin/clang -c -o /tmp/rffi/.lake/build/ir/Rffi/Basic.c.o.export /tmp/rffi/.lake/build/ir/Rffi/Basic.c -I /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1/include -fstack-clash-protection -fwrapv -fPIC -fvisibility=hidden -Wno-unused-command-line-argument --sysroot /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1 -nostdinc -isystem /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1/include/clang -O3 -DNDEBUG -D

info: Version 4.1.2 of elan is available! Use `elan self update` to update.
info: downloading https://github.com/leanprover/lean4/releases/download/v4.20.1/lean-4.20.0-linux.tar.zst
347.5 MiB / 347.5 MiB (100 %)  10.4 MiB/s ETA:   0 s
info: installing /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1
ld.lld: error: undefined symbol: main
>>> referenced by start.S:104 (../sysdeps/x86_64/start.S:104)
>>>               /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1/lib/Scrt1.o:(_start)
clang: error: linker command failed with exit code 1 (use -v to see invocation)


ModuleNotFoundError: No module named 'leandojo'

In [ ]:
.global myforty
myforty:
movq 42, %rax
ret

In [1]:
import cffi
ffi = cffi.FFI()
ffi.cdef("uint32_t myforty();")
lib = ffi.dlopen("/tmp/fortytwo.so")
print(43490)
lib.myforty()

: 